# EE 564 Project 2

## Tesla Model S Motor Design


[Tesla Model S](http://www.teslamotors.com/models)

[RWD 85 Model](http://en.wikipedia.org/wiki/Tesla_Model_S#Powertrain)

* Max. Power: 360 hp (270 kW)
* Max. Torque: 441 Nm
* Top Speed: 225 km/h 

[Specs1](http://teslatap.com/undocumented/) and [Specs2](http://my.teslamotors.com/it_IT/forum/forums/tesla-model-s-motor-output)

**Useful Links:**

* [How does Tesla make their motors in the model S so small?](http://www.reddit.com/r/AskEngineers/comments/2mv2h5/how_does_tesla_make_their_motors_in_the_model_s/)

* [Tesla Motor Technology](http://my.teslamotors.com/roadster/technology/motor)

* [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf)

## Design Specifications

From [Specs1](http://teslatap.com/undocumented/): "Motor Inverter – This converts the DC battery voltage to AC, powering the motor. It delivers up to 1000 amps (from 2010 10K annual report)."

In [100]:
import math
Pn=270e3 # Max. power (W)
Iline=1000 # Max. current (A)
Vll=Pn/(math.sqrt(3)*Iline) # Max. L-L voltage (V)
Vln=Vll/math.sqrt(3) # Max. L-N voltage (V)
print Vll
print Vln

155.884572681
90.0


In [101]:
Vll=200 # Design motor with 200 V rating
Vln=200/math.sqrt(3)
f1=60 # Supply frequency (Hz)
m=3 # Number of phases, Star-connected
nn=0.96 # Target efficiency
cosphin=0.83 # Target power factor

From [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf);

In [102]:
p1=2 # Number of pole pairs

## Main Dimensions of Stator Core

Reference: I. Boldea and S.A. Nasar, "The Induction Machine Handbook", CRC Press LLC, 2002.

Equation, figure and table numbers are given as comments next to the equation.

In [103]:
KE=0.98-0.005*p1 # Eq. 15.1
Sgap=KE*Pn/(nn*cosphin) # Apparent power (VA), Eq. 15.2
print KE
print Sgap

0.97
328689.759036


In [104]:
C0=250e3 # Esson's constant (J/m^3), Fig. 14.14
lambda1=1.5 # Stack aspect ratio, Table 15.1
Dis=math.pow((2*p1*p1*Sgap)/(math.pi*lambda1*f1*C0),1.0/3.0) # Stator bore diameter (m), Eq. 15.1
print Dis

0.333821719418


In [105]:
tau=math.pi*Dis/(2*p1) # Pole pitch (m), Eq. 15.2
L=lambda1*tau # Stack length (m), Eq. 15.2
print tau
print L

0.262182965333
0.393274447999


From [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf);

In [106]:
q=4 # Number of stator slots per pole per phase
taus=tau/(m*q) # Slot pitch (m), Eq. 15.3
print taus

0.0218485804444


In [107]:
KD=0.62 # Dis/Dout, Table 15.2
Dout=Dis/KD # Stator outer diameter (m), Eq. 15.4
print Dout

0.538422128093


In [108]:
g=(0.1+0.012*math.pow(Pn,1.0/3.0))*1e-3 # Air gap length (m), Eq. 15.5
print g

0.000875596488411


## Stator Winding

In [109]:
Ns=2*p1*q*m # Number of stator slots, Eq. 15.6
print Ns

48


Two-layer winding with chorded coils, y/$\tau$=10/12 is chosen as 10/12=0.83 is close to 0.8, which would reduce the first (5$^{th}$ order) stator mmf space harmonic.

In [110]:
alphaec=2*math.pi*p1/Ns # Electrical angle between emfs in neighboring slots (rad), Eq. 15.7
print alphaec

0.261799387799


The distribution of phases in slots will be given by MotorAnalysis software later.

In [111]:
Kq1=math.sin(q*alphaec/2)/(q*math.sin(alphaec/2)) # Distribution factor, Eq. 15.8
print Kq1

0.957662196943


In [112]:
Ky1=math.sin(math.pi/2*10/12) # Pitch factor, Eq. 15.9
print Ky1

0.965925826289


In [113]:
Kw1=Kq1*Ky1 # Winding factor
print Kw1

0.925030648888


In [114]:
Bg=0.65 # Recommended air gap flux density (T), Eq. 15.11
Kf=1.085
alphai=0.729 # Pole spanning coefficient, Fig. 14.13
phi=alphai*tau*L*Bg # Pole flux (Wb), Eq. 15.10
print phi


0.0488586076188


In [115]:
W1=KE*Vln/(4*Kf*Kw1*f1*phi) # Number of turns per phase, Eq. 15.12
print W1

9.51706233954


In [116]:
a1=1 # Number of current paths in parallel
ns=a1*W1/(p1*q) # Number of conductors per slot, Eq. 15.13
print ns

1.18963279244


This should be an even number since there are two distinct coils per slot in a double layer winding.

In [117]:
ns=2
W1=p1*q*ns
print W1

16


In [118]:
Bg=Bg*9.517/16 # Actual air gap flux density (T), Eq. 15.15
print Bg

0.386628125


In [119]:
Iln=Pn/(3*Vln*cosphin*nn) # Rated current (A), Eq. 15.16
print Iln

978.191344636


In [120]:
Jcos=7 # Current density (A/mm^2), Eq. 15.17
Aco=Iln/(Jcos*a1) # Magnetic wire cross section (mm^2), Eq. 15.18
print Aco

139.741620662


In [122]:
dco=math.sqrt(4*Aco/math.pi) # Wire gauge diameter (mm), Eq. 15.19
print dco

13.3388364362
2.66776728725


We can use few conductors in parallel, $a_p$=5.

In [128]:
ap=10
dco=math.sqrt(4*Aco/(math.pi*ap)) # Wire gauge diameter (mm), Eq. 15.19
print dco

4.21811044749


In [24]:
A1=C0/(Kf*alphai*Kw1*3.14*Bg*2*p1)
print A1

38464.3700122


# Stator Slot Sizing

In [25]:
Kfill=0.44
dco=1.32
ap=5
Asu=math.pi*math.pow(dco,2)*ap*ns/(4*Kfill)
print Asu

124.407069082


In [26]:
print taus
print 8*g
Kfe=0.96
Bts=1.55
bts=Bg*taus*L/(Bts*L*Kfe)
print bts

0.0291215941612
0.00700477190729
0.0141991419784


In [27]:
bos=2.2e-3
hos=1e-3
hw=1.5e-3
bs1=math.pi*(Dis+2*hos+2*hw)/Ns-bts
print bs1

0.0153735553912


In [28]:
bs2=math.sqrt(4*Asu*math.tan(math.pi/Ns)+math.pow(bs1,2))*1e-3
print bs2

0.0065982625876


In [29]:
hs=2*Asu/(bs1+bs2)*1e-6
print hs

0.0113242399152


In [30]:
Fmg=1.2*g*Bg/(4*math.pi*1e-7)
print Fmg

606.631951354


In [31]:
Hts=1760
Fmts=Hts*(hs+hos+hw)
print Fmts

24.3306622507


In [32]:
Kst=1.4-1
Fmtr=Kst*Fmg-Fmts
print Fmtr

218.322118291


In [33]:
hcs=(Dout-(Dis+2*(hos+hw+hs)))/2
print hcs

0.0884932575487


In [34]:
Bcs=phi/(2*L*hcs)
print Bcs

0.755688903364


# Rotor Slots

In [35]:
print Ns

36


In [36]:
Nr=28
K1=0.8*cosphin+0.2
Ib=K1*(2*m*W1*Kw1)/Nr*Iln
print Ib

7838.92674395


In [37]:
Jb=4
Ab=Ib/Jb*1e-6
print Ab

0.00195973168599


In [38]:
Icr=Ib/(2*math.sin(math.pi*p1/Nr))
print Icr

17613.9085087


In [39]:
Jcr=0.75*Jb*1e6
Acr=Icr/Jcr
print Acr

0.00587130283623


In [40]:
hor=0.5e-3
bor=1.5e-3
taur=math.pi*(Dis-2*g)/Nr
print taur

0.0372645573945


In [41]:
Btr=1.60
btr=Bg/(Kfe*Btr)*taur
print btr

0.0176017009991


In [42]:
print Dis
Dre=Dis-g
print Dre
d1=(math.pi*(Dre-2*hor)-Nr*btr)/(math.pi+Nr)
print d1

0.333878149619
0.333002553131
0.017666699325
